In [1]:
from carbon.helpers.stdimports import *
from carbon.helpers.soltest import SolTestBase
from carbon.helpers.floatint import *
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(SolTestBase))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonFloatInt32))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonOrderUI))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(P))
from math import log2, floor, ceil, sqrt
print_version(require="2.3.2")

[stdimports] imported np, pd, plt, os, sqrt, exp, log
SolTestBase v0.2 (11/02/2023)
CarbonFloatInt32 v1.0 (10/Feb/2023)
CarbonOrderUI v1.7 (11/Feb/2023)
CarbonPair v2.2 (11/Feb/2022)
Carbon v2.3.2-BETA4 (11/Feb/2022)


# Solidity Curve Testing (NBTest058)

In the terminology in this sheet we consider by source and by target **from the point of view of the AMM**, therefore _by source_ fixes _dy_ and calculates _dx_ and vice versa.

**By source (dx from dy)**

$$
\Delta x = \frac{\Delta y z^2}{(Ay+Bz)(Ay+Bz-A\Delta y)} =_{A=0} \frac{\Delta y} {B^2}
$$



**By target (dy from dx)**

$$
\Delta y = \frac{\Delta x(Ay+Bz)^2}{A\Delta x(Ay+Bz)+z^2} =_{A=0} \frac{\Delta x(Bz)^2}{z^2} = \Delta x B^2
$$

[doc][doc]


[doc]:https://docs.google.com/document/d/1x4ZbbS3nIRSJ0ojaOrcBhOc-_eOTga6BajDjK45o7u4/edit

In [2]:
FI32 = CarbonFloatInt32
FI40 = CarbonFloatInt40

## Demo and test of yzABS in CarbonOrderUI and pair decimals

In [3]:
ETHUSDC = P("ETH/USDC").sd(18,6)
assert ETHUSDC.decimals == {'ETH': 18, 'USDC': 6, '_TKNB': 18, '_TKNQ': 6, '_DIFFQB': -12}
assert ETHUSDC.decdiffqb == -12
assert P(ETHUSDC).has_decimals
assert ETHUSDC.price_convention == 'USDC per ETH'
ETHUSDC

P('ETH/USDC').sd(18,6)

In [4]:
oui = CarbonOrderUI.from_prices(ETHUSDC, "ETH", 1500, 2000, 1, 0.5)
r = oui.yzABS(sx=10, verbose=True)
ddf, ddf2 = 10**12, 10**6
assert r.y == 0.5 * 10**18
assert r.z == 10**18
assert r.A == int(oui.S * ddf2 * r.S)
assert r.B == int(oui.B * ddf2 * r.S)
assert r.S == 2**10
assert oui.descr() == 'Sell ETH buy USDC from 1500.0000 to 2000.0000 USDC per ETH'

[yzABS] pair=P('ETH/USDC').sd(18,6), y=ETH(18), x=USDC(6)
[yzABS] scale = 2**10 = 1024
[yzABS] y=0.5 -> y_wei=5e+17 [ETH-wei]
[yzABS] yint=1 -> z_wei=1000000000000000000 [ETH-wei]
[yzABS] pa_raw=0.0006666666666666666 ETH per USDC -> 666666666 ETH-wei per USDC-wei
[yzABS] pb_raw=0.0005 ETH per USDC -> 500000000 ETH-wei per USDC-wei
[yzABS] a=0.0034592091997182155 -> 3459.2091997182156 * scale = 3542230
[yzABS] b=0.022360679774997897 -> 22360.679774997898 * scale = 22897336
[yzABS] yzABS = r(y=5e+17, z=1000000000000000000, A=3542230, B=22897336, S=1024)


In [5]:
oui = CarbonOrderUI.from_prices(ETHUSDC, "USDC", 1000, 500, 1100, 790)
r = oui.yzABS(sx=40, verbose=True)
ddf, ddf2 = 10**-12, 10**-6
assert r.y == 790 * 10**6
assert r.z == 1100 * 10**6
assert r.A == int(oui.S * ddf2 * r.S)
assert r.B == int(oui.B * ddf2 * r.S)
assert r.S == 2**40

[yzABS] pair=P('ETH/USDC').sd(18,6), y=USDC(6), x=ETH(18)
[yzABS] scale = 2**40 = 1099511627776
[yzABS] y=790 -> y_wei=790000000 [USDC-wei]
[yzABS] yint=1100 -> z_wei=1100000000 [USDC-wei]
[yzABS] pa_raw=1000.0 USDC per ETH -> 0 USDC-wei per ETH-wei
[yzABS] pb_raw=500.00000000000006 USDC per ETH -> 0 USDC-wei per ETH-wei
[yzABS] a=9.262096826685895 -> 9.262096826685894e-06 * scale = 10183783
[yzABS] b=22.360679774997898 -> 2.2360679774997898e-05 * scale = 24585827
[yzABS] yzABS = r(y=790000000, z=1100000000, A=10183783, B=24585827, S=1099511627776)


## Demo [NOTEST]

### Trade functions

In [6]:
mulDivF = lambda x, y, z: (x * y) // z
mulDiv = mulDivF
mulDivC = lambda x, y, z: (x * y + z - 1) // z

**this is actually by target**

In [7]:
def trade_by_target_act(params, C):

    dx = params[0]
    y,z,A,B,s = params[1]
    print(params)
    ONE = s
    temp1 = C(y * A + z * B, "temp1")               # 177 bits at most; cannot overflow
    temp2 = C(temp1 * dx / ONE, "temp2")            # 224 bits at most; can overflow; some precision loss
    temp3 = C(temp2 * A + z * z * ONE, "temp3")     # 256 bits at most; can overflow
    dy = mulDiv(temp1, temp2, temp3)
    print(dx, temp1, temp2, temp3, dy)
    return dy

In [8]:
# # by target_act

# temp1 = z * ONE
# temp2 = y * A + z * B
# temp3 = temp2 * dy
# scale = mulDivC(temp3, A, 2**256-1)
# temp4 = mulDivC(temp1, temp1, scale)
# temp5 = mulDivC(temp3, A, scale)
# dx    = mulDivF(temp2, temp3 // scale, temp4 + temp5)

**those are actually by source**

this one is SKL version of the fixed code

In [9]:
def trade_by_source_act(params, C):

    dy = params[0]
    y,z,A,B,s = params[1]
    print(params)
    ONE = s
    temp1 = C(z * ONE, "temp1")                  
    temp2 = C(y * A + z * B, "temp2")         
    temp3 = C(temp2 - dy * A, "temp3")        
    scale = mulDiv(temp2, temp3, 2**255)+1
    temp1s = C(temp1//scale, "temp1s")
    temp2s = C(temp2//scale, "temp2s")
    dx = mulDiv(
        C(dy*temp1s, "dx*temp1s"), 
        temp1, 
        C(temp2s*temp3, "temp2s*temp3")
    )
    print(dy, dx*temp1s, temp1, temp2s*temp3, dx)
    return dx

this is BM version of the fixed code

In [10]:
# # by source_act
# mulDivF = lambda x, y, z: x * y // z
# mulDivC = lambda x, y, z: (x * y + z - 1) // z

# temp1 = z * ONE
# temp2 = y * A + z * B
# temp3 = temp2 - dx * A
# scale = mulDivC(temp2, temp3, 2**256-1)
# temp4 = mulDivC(temp1, temp1, scale)
# temp5 = mulDivF(temp2, temp3, scale)
# dy    = mulDivC(dx, temp4, temp5)

that's the previous, failing, version of the code

In [11]:
def trade_by_source_old_act(params, C):

    dy = params[0]
    y,z,A,B,s = params[1]
    ONE = s
    temp1 = C(z * ONE, "temp1")                  
    temp2 = C(y * A + z * B, "temp2")         
    temp3 = C(temp2 - dy * A, "temp3")        
    dx = mulDiv(
        C(dy*temp1, "dy*temp1"), 
        temp1, 
        C(temp2*temp3, "temp2*temp3")
    ) 
    print(dy, dy*temp1, temp1, temp2*temp3, dx)
    return dx

### Analysis

In [12]:
class STB(SolTestBase):
    #PRINT_LVL_DEFAULT = SolTestBase.LVL_LOG
    PRINT_LVL_DEFAULT = SolTestBase.LVL_WARN
VERBOSE = False

#### TKN/DAI -- same decimality, same reasonable price

In [13]:
TKNDAI = P("TKN/DAI").sd(18,18)
price = 5  # DAI per TKN
oui = CarbonOrderUI.from_prices(TKNDAI, "TKN", price, price, 1000, 1000)
c = curve = oui.yzABS(sx=40, verbose=VERBOSE)

In [14]:
params_bysrc  = (1e18, curve) # dy = wei sent (by source, 1 TKN)
dx = trade_by_source_act( params_bysrc, STB(context=("by_source", curve)))
dx/1e18 # 1 TKN -> 5 DAI ==> 5 DAI per TKN

(1e+18, r(y=1000000000000000000000, z=1000000000000000000000, A=0, B=491716548351, S=1099511627776))
1e+18 5497558138896018159282952339456000000000000000000000 1099511627776000000000000000000000 241785163922221334256682292088662337232306176000000000000000000000 5000000000014568431


5.000000000014569

In [15]:
params_bytarg = (5e18, curve)       # dx = token wei received (target, DAI)
dy = trade_by_target_act( params_bytarg, STB(context=("by_target", curve)) )
dy/1e18 # 1 TKN ->  5 DAI --> 5 DAI per TKN

(5e+18, r(y=1000000000000000000000, z=1000000000000000000000, A=0, B=491716548351, S=1099511627776))
5e+18 491716548351000000000000000000000 2236067977496531949120054474223665545216 1099511627776000000000000000000000000000000000000000000 999999999997086190


0.9999999999970862

#### ETH/USDC -- y=ETH, x=USDC

In [16]:
ETHUSDC = P("ETH/USDC").sd(18,6)
oui = CarbonOrderUI.from_prices(ETHUSDC, "ETH", 2000, 2000, 1, 0.5)
curve = oui.yzABS(sx=40, verbose=VERBOSE)

In [17]:
params_bysrc  = (1*1e18, curve) # dy = token wei sent (by source, 1 ETH)
dx = trade_by_source_act( params_bysrc, STB(context=("by_source", curve)))
dx/1e6 # 1 ETH -> 2000 USDC

(1e+18, r(y=5e+17, z=1000000000000000000, A=0, B=24585827417585820, S=1099511627776))
1e+18 2199023254452488372224000000000000000000 1099511627776000000000000000000 604462909807314618688415159765465413263269547110658276245826332262400 1999999999


1999.999999

In [18]:
params_bytarg = (2000*1e6, curve)       # dx = token wei received (target, USDC)
dy = trade_by_target_act( params_bytarg, STB(context=("by_target", curve)) )
dy/1e18 # 1 ETH -> 2000 USDC

(2000000000.0, r(y=5e+17, z=1000000000000000000, A=0, B=24585827417585820, S=1099511627776))
2000000000.0 24585827417585819755229268861255680 44721359549995793980834690629632 1099511627776000000000000000000000000000000000000 1000000000000000027


1.0

#### ETH/USDC -- y=USDC, x=ETH

In [19]:
oui = CarbonOrderUI.from_prices(ETHUSDC, "USDC", 1000, 1000, 750, 750)
curve = oui.yzABS(sx=40, verbose=VERBOSE)
print(curve)

r(y=750000000, z=750000000, A=0, B=34769610, S=1099511627776)


In [20]:
params_bysrc  = (1000*1e6, curve)       # dy = USDC-wei
dx = trade_by_source_act( params_bysrc, STB(context=("by_source", curve)) )
dx/1e18 # 1000 USDC -> 1 ETH

(1000000000.0, r(y=750000000, z=750000000, A=0, B=34769610, S=1099511627776))
1000000000.0 824633748159494423332333289472000000000 824633720832000000000 680020750998056250000000000000000 1000000033138948521


1.0000000331389485

In [21]:
params_bytarg = (1e18, curve)  # dx = ETH-wei
dy = trade_by_target_act( params_bytarg, STB(context=("by_target", curve)) )
dy/1e6 # 1000 USDC -> 1 ETH

(1e+18, r(y=750000000, z=750000000, A=0, B=34769610, S=1099511627776))
1e+18 26077207500000000 23717082058283267653632 618475290624000000000000000000 999999966


999.999966

#### SHIB/USDC -- y=SHIB, x=USDC

In [22]:
SHIBUSDC = P("SHIB/USDC").sd(18,6)
price = 1e-5
oui = CarbonOrderUI.from_prices(SHIBUSDC, "SHIB", price, price, 2e5, 2e5)
curve = oui.yzABS(sx=40, verbose=VERBOSE)
print(curve)

r(y=2e+23, z=2e+23, A=0, B=347696105761141489664, S=1099511627776)


In [23]:
params_bysrc  = (1*1e5*1e18, curve)      # dy = SHIB-wei (1 USD worth of SHIB)
dx = trade_by_source_act( params_bysrc, STB(context=("by_source", curve)) )
dx / 1e6 # 1e5 SHIB -> 1 USDC

(1e+23, r(y=2e+23, z=2e+23, A=0, B=347696105761141489664, S=1099511627776))
[WARNING:OVERFLOW:TEMP2S*TEMP3] 255 bits: close to max
1e+23 2632807291694421164755057000000 219902325555199981553255926290448384 57896044618229089136646689796713055452480359437934408123369084530614903767040 1000000


1.0

In [24]:
params_bytarg = (1*1e6, curve) # dx = USDC-wei
dy = trade_by_target_act( params_bytarg, STB(context=("by_target", curve)) )
dy/1e18 # 1e5 SHIB -> 1 USDC

(1000000.0, r(y=2e+23, z=2e+23, A=0, B=347696105761141489664, S=1099511627776))
1000000.0 69539221152228296189580545547731059544686592 63245553203367578540979698731480776704 43980465111039994123099750822391432276334879391719429767168 99999999999999985943809


99999.99999999999

#### Nick's example DAI/USDC -- Selling DAI

In [25]:
DAIUSDC = P("DAI/USDC").sd(18,6)
price = 1
oui = CarbonOrderUI.from_prices(DAIUSDC, "DAI", price, price, 1e5, 1e5)
curve = oui.yzABS(sx=40, verbose=VERBOSE)
print(curve)

r(y=1e+23, z=1e+23, A=0, B=1099511627776000000, S=1099511627776)


In [26]:
params_bysrc  = (1*1e18, curve)        # dy = DAI-wei
dx = trade_by_source_act( params_bysrc, STB(context=("by_source", curve)) )
dx/1e6 # 1 DAI -> 1 USDC

(1e+18, r(y=1e+23, z=1e+23, A=0, B=1099511627776000000, S=1099511627776))
[WARNING:OVERFLOW:TEMP2S*TEMP3] 255 bits: close to max
1e+18 526560283637935027904014110996239763 109951162777599990776627963145224192 57895973354467169559526534691796259748754773305955415093504382476762774241280 999999


0.999999

In [27]:
params_bytarg = (1*1e6, curve)  # dx = USDC-wei
dy = trade_by_target_act( params_bytarg, STB(context=("by_target", curve)) )
dy / 1e18 # 1 DAI -> 1 USDC

(1000000.0, r(y=1e+23, z=1e+23, A=0, B=1099511627776000000, S=1099511627776))
1000000.0 109951162777599990580649754106133915631616 99999999999999996863366107917975552 10995116277759998530774937705597858069083719847929857441792 1000000000000000016


1.0

#### Nick's example DAI/USDC -- Selling USDC

In [28]:
SHIBUSDC = P("DAI/USDC").sd(18,6)
price = 1
oui = CarbonOrderUI.from_prices(SHIBUSDC, "USDC", price, price, 1e5, 1e5)
curve = oui.yzABS(sx=40, verbose=VERBOSE)

In [29]:
params_bysrc  = (1*1e6, curve)       # dy = USDC-wei
dx = trade_by_source_act( params_bysrc, STB(context=("by_source", curve)) )
dx/1e18

(1000000.0, r(y=100000000000.0, z=100000000000.0, A=0, B=1099511, S=1099511627776))
1000000.0 109951288332907530323650989260800000000000 109951162777600000000000 12089244391210000000000000000000000 1000001141918869783


1.0000011419188697

In [30]:
params_bytarg = (1e18, curve)      # dx = DAI-wei
dy = trade_by_target_act( params_bytarg, STB(context=("by_target", curve)) )
dy/1e6

(1e+18, r(y=100000000000.0, z=100000000000.0, A=0, B=1099511, S=1099511627776))
1e+18 109951100000000000 99999942904105404465152 10995116277760000000000000000000000 999998


0.999998

## SolTestBase tests

In [31]:
TB0 = SolTestBase()
TBrw = SolTestBase(raise_lvl=TB0.LVL_WARN)
TBre = SolTestBase(raise_lvl=TB0.LVL_ERR)

In [32]:
try:
    TBrw.check_uint256(1, "testlabel")
    raise RuntimeError("should raise")
except TB0.UnderflowError as e:
    print(e)
try:
    TBrw.check_uint256(2000, "testlabel")
    raise RuntimeError("should raise")
except TB0.UnderflowWarning as e:
    print(e)
try:
    TBrw.check_uint256(2**254, "testlabel")
    raise RuntimeError("should raise")
except TB0.OverflowWarning as e:
    print(e)
try:
    TBrw.check_uint256(2**256-1, "testlabel")
    raise RuntimeError("should raise")
except TB0.OverflowWarning as e:
    print(e)
try:
    TBrw.check_uint256(2**256, "testlabel")
    raise RuntimeError("should raise")
except TB0.OverflowError as e:
    print(e)

[ERROR:UNDERFLOW:TESTLABEL] 1 bits: < underflow threshold
('[ERROR:UNDERFLOW:TESTLABEL] 1 bits: < underflow threshold', {}, None)
[WARNING:UNDERFLOW:TESTLABEL] 11 bits: underflow warning
('[WARNING:UNDERFLOW:TESTLABEL] 11 bits: underflow warning', {}, None)
[WARNING:OVERFLOW:TESTLABEL] 254 bits: close to max
('[WARNING:OVERFLOW:TESTLABEL] 254 bits: close to max', {}, None)
[WARNING:OVERFLOW:TESTLABEL] 256 bits: close to max
('[WARNING:OVERFLOW:TESTLABEL] 256 bits: close to max', {}, None)
[ERROR:OVERFLOW:TESTLABEL] 256 bits: > max
('[ERROR:OVERFLOW:TESTLABEL] 256 bits: > max', {}, None)


In [33]:
try:
    TBre.check_uint256(1, label="testlabel")
    raise RuntimeError("should raise")
except TBre.UnderflowError as e:
    print(e)
assert TBre.check_uint256(2000, label="testlabel") == 2000
assert TBre.check_uint256(2**254, label="testlabel") == 2**254
assert TBre.check_uint256(2**256-1, label="testlabel") == 2**256-1
try:
    TBre.check_uint256(2**256, label="testlabel")
    raise RuntimeError("should raise")
except TBre.OverflowError as e:
    print(e)

[ERROR:UNDERFLOW:TESTLABEL] 1 bits: < underflow threshold
('[ERROR:UNDERFLOW:TESTLABEL] 1 bits: < underflow threshold', {}, None)
[WARNING:UNDERFLOW:TESTLABEL] 11 bits: underflow warning
[WARNING:OVERFLOW:TESTLABEL] 254 bits: close to max
[WARNING:OVERFLOW:TESTLABEL] 256 bits: close to max
[ERROR:OVERFLOW:TESTLABEL] 256 bits: > max
('[ERROR:OVERFLOW:TESTLABEL] 256 bits: > max', {}, None)


In [34]:
assert TB0.check_uint256(1, log_f=TB0.print_f) == 1
assert TB0.check_uint256(2000, log_f=TB0.print_f) == 2000
assert TB0.check_uint256(2**254, log_f=TB0.print_f) == 2**254
assert TB0.check_uint256(2**256-1, log_f=TB0.print_f) == 2**256-1
assert TB0.check_uint256(2**256, log_f=TB0.print_f) == 2**256

[ERROR:UNDERFLOW:] 1 bits: < underflow threshold
[WARNING:UNDERFLOW:] 11 bits: underflow warning
[WARNING:OVERFLOW:] 254 bits: close to max
[WARNING:OVERFLOW:] 256 bits: close to max
[ERROR:OVERFLOW:] 256 bits: > max


In [35]:
assert TB0._logmsg(level=TB0.LVL_WARN, isoverflow=True, label="1", msg="") == '[WARNING:OVERFLOW:1] '
assert TB0._logmsg(TB0.LVL_ERR, False, "mylabel", "mymessage") == '[ERROR:UNDERFLOW:MYLABEL] mymessage'
TB0.print_f(False, False, "mylabel", "mymessage")

In [36]:
assert TB0.bindig(0) == 0
assert TB0.bindig(1) == 1
assert TB0.bindig(2) == 2
assert TB0.bindig(3) == 2
assert TB0.bindig(2**10) == 11
assert TB0.bindig(2**10+1) == 11
assert TB0.bindig(2**10-1) == 10